<h2 align="center">Codebasics DL Course: Vehicle Damage Detection Project: Hyperparameter Tunning</h2>

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import optuna

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load Data

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
dataset_path = "./dataset"

dataset = datasets.ImageFolder(root=dataset_path, transform=image_transforms)
len(dataset)

2300

In [5]:
2300*0.75

1725.0

In [6]:
class_names = dataset.classes
class_names 

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

6

In [8]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1725, 575)

In [9]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

### Model Training & Hyperparameter Tunning

In [11]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [12]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    # Load the model
    model = CarClassifierResNet(num_classes=num_classes, dropout_rate=dropout_rate).to(device)
    
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    
    # Training loop (using fewer epochs for faster hyperparameter tuning)
    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Report intermediate result to Optuna
        trial.report(accuracy, epoch)
        
        # Handle pruning (if applicable)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time: {end - start} seconds")
    
    return accuracy

In [13]:
# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-05-14 12:59:18,018] A new study created in memory with name: no-name-4db78303-3d1c-482c-899b-9355cce283f3
[I 2025-05-14 13:03:43,615] Trial 0 finished with value: 75.47826086956522 and parameters: {'lr': 0.002637256149720464, 'dropout_rate': 0.3009606953555503}. Best is trial 0 with value: 75.47826086956522.


Execution time: 265.0219016075134 seconds


[I 2025-05-14 13:08:11,270] Trial 1 finished with value: 75.47826086956522 and parameters: {'lr': 7.679338121036558e-05, 'dropout_rate': 0.40699327854512357}. Best is trial 0 with value: 75.47826086956522.


Execution time: 267.23169112205505 seconds


[I 2025-05-14 13:12:36,109] Trial 2 finished with value: 77.21739130434783 and parameters: {'lr': 0.00017990143751747141, 'dropout_rate': 0.6357946197012911}. Best is trial 2 with value: 77.21739130434783.


...

[I 2025-05-14 14:48:35,030] Trial 15 finished with value: 78.6086956521739 and parameters: {'lr': 0.00046044263599502686, 'dropout_rate': 0.23925514041227294}. Best is trial 12 with value: 78.78260869565217.


Execution time: 2037.1821212768555 seconds


[I 2025-05-14 14:49:58,247] Trial 16 pruned. 
[I 2025-05-14 14:51:32,429] Trial 17 pruned. 
[I 2025-05-14 14:56:18,186] Trial 18 finished with value: 79.30434782608695 and parameters: {'lr': 0.0024742849948221073, 'dropout_rate': 0.2017476571477562}. Best is trial 18 with value: 79.30434782608695.


Execution time: 285.22234320640564 seconds


[I 2025-05-14 14:57:53,513] Trial 19 pruned. 


In [14]:
study.best_params

{'lr': 0.0024742849948221073, 'dropout_rate': 0.2017476571477562}

In the 18th run it gave me best results with lr=0.002 and dropout=0.2. In the second run it gave different results as you see it above. Both the runs are not making much difference in terms of accuracy hence I will just use the results from the first run